In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class Taxitrip:
  def __init__(self,csv_path_file):
    self.df = pd.read_csv(csv_path_file)

  def get_data(self):
    return self.df

  def show_info(self):
    return self.df.info(),self.df.isnull().sum()

  def fill_nan(self):
    cols = ['VendorID','passenger_count','RatecodeID','payment_type']
    for col in cols:
      self.df[col].fillna(self.df[col].mode()[0],inplace=True)
    self.df['total_amount'].fillna(self.df['total_amount'].mean(),inplace=True)
    self.df['store_and_fwd_flag'].fillna(self.df['store_and_fwd_flag'].mode()[0],inplace=True)
    self.df['congestion_surcharge'].fillna(self.df['congestion_surcharge'].mean(),inplace=True)
    self.df['trip_distance'].fillna(self.df['trip_distance'].mean(),inplace=True)
    self.df['extra'].fillna(self.df['extra'].mean(),inplace=True)
    self.df['mta_tax'].fillna(self.df['mta_tax'].mean(),inplace=True)
    self.df['tip_amount'].fillna(self.df['tip_amount'].mean(),inplace=True)
    self.df['improvement_surcharge'].fillna(self.df['improvement_surcharge'].mean(),inplace=True)
    self.df['fare_amount'].fillna(self.df['fare_amount'].mean(),inplace=True)
    self.df['tolls_amount'].fillna(self.df['tolls_amount'].mean(),inplace=True)
    self.df['PULocationID'].fillna(self.df['PULocationID'].mode()[0],inplace=True)
    self.df['DOLocationID'].fillna(self.df['DOLocationID'].mode()[0],inplace=True)
    self.df['tpep_dropoff_datetime'].fillna(self.df['tpep_dropoff_datetime'].mode()[0],inplace=True)
    return self.df

  def calculate_trip_duration(self):
    self.df['tpep_pickup_datetime'] = pd.to_datetime(self.df['tpep_pickup_datetime'], errors='coerce')
    self.df['tpep_dropoff_datetime'] = pd.to_datetime(self.df['tpep_dropoff_datetime'], errors='coerce')
    self.df['trip_duration'] = (self.df['tpep_dropoff_datetime'] - self.df['tpep_pickup_datetime'])
    self.df['trip_duration_minutes'] = (self.df['trip_duration'].dt.total_seconds() / 60)
    self.df.loc[self.df['trip_duration_minutes'] < 0, 'trip_duration_minutes'] = None
    return self.df[['tpep_pickup_datetime', 'tpep_dropoff_datetime','trip_duration', 'trip_duration_minutes']]

  def payment_type(self):
    return self.df.groupby('payment_type')['fare_amount'].mean()

  def longest_trip_distance(self):
    return self.df.loc[self.df['trip_distance'] == self.df['trip_distance'].max()]

  def shortest_trip_distance(self):
    return self.df.loc[self.df['trip_distance'] == self.df['trip_distance'].min()]

  def most_popular_pickup_location(self):
    return self.df['PULocationID'].value_counts().idxmax()

  def most_popular_dropoff_location(self):
    return self.df['DOLocationID'].value_counts().idxmax()

  def most_popular_payment_type(self):
    return self.df['payment_type'].value_counts().idxmax()

  def maximum_tip_amount(self):
    return self.df.loc[self.df['tip_amount'] == self.df['tip_amount'].max()]

  def average_trip_duration(self):
    self.df['tpep_pickup_datetime'] = pd.to_datetime(self.df['tpep_pickup_datetime'], errors='coerce')
    self.df['tpep_dropoff_datetime'] = pd.to_datetime(self.df['tpep_dropoff_datetime'], errors='coerce')
    self.df['trip_duration'] = (self.df['tpep_dropoff_datetime'] - self.df['tpep_pickup_datetime'])
    self.df['trip_duration_minutes'] = (self.df['trip_duration'].dt.total_seconds() / 60)
    return self.df['trip_duration_minutes'].mean()

  def average_trip_distance(self):
    return self.df['trip_distance'].mean()

  def average_fare_amount(self):
    return self.df['fare_amount'].mean()

  def average_tip_amount(self):
    return self.df['tip_amount'].mean()

  def average_total_amount(self):
    return self.df['total_amount'].mean()

  def average_congestion_surcharge(self):
    return self.df['congestion_surcharge'].mean()

  def average_mta_tax(self):
    return self.df['mta_tax'].mean()

  def common_trip(self):
    return self.df['trip_distance'].mode()[0]

  def drop_loc_highest_fare(self):
    return self.df.sort_values(by='fare_amount',ascending=False ).head(1)[['DOLocationID','fare_amount']]

  def correlation(self):
    correlation = self.df['trip_distance'].corr(self.df['fare_amount'])
    return correlation

  def fraud_trips(self):
    fraud_trips = self.df[(self.df['fare_amount'] > 30) & (self.df['trip_distance'] < 1)]
    return fraud_trips.shape[0]

  def plot_trip_distance_distribution(self):
    plt.figure(figsize=(10, 6))
    sns.histplot(self.df['trip_distance'], bins=50, kde=True)
    plt.title('Distribution of Trip Distance')
    plt.xlabel('Trip Distance (miles)')
    plt.ylabel('Frequency')
    plt.show()

  def plot_fare_amount_distribution(self):
    plt.figure(figsize=(10, 6))
    sns.histplot(self.df['fare_amount'], bins=50, kde=True, color='green')
    plt.title('Distribution of Fare Amount')
    plt.xlabel('Fare Amount ($)')
    plt.ylabel('Frequency')
    plt.show()

  def plot_payment_type_counts(self):
    plt.figure(figsize=(8, 5))
    sns.countplot(x='payment_type', data=self.df)
    plt.title('Payment Type Counts')
    plt.xlabel('Payment Type')
    plt.ylabel('Count')
    plt.show()

  def plot_trip_duration_vs_fare(self):
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x='trip_duration_minutes', y='fare_amount', data=self.df, alpha=0.5)
    plt.title('Trip Duration vs Fare Amount')
    plt.xlabel('Trip Duration (minutes)')
    plt.ylabel('Fare Amount ($)')
    plt.show()

  def plot_tip_amount_distribution(self):
    plt.figure(figsize=(10, 6))
    sns.histplot(self.df['tip_amount'], bins=40, kde=True, color='orange')
    plt.title('Distribution of Tip Amount')
    plt.xlabel('Tip Amount ($)')
    plt.ylabel('Frequency')
    plt.show()

  def plot_top_pickup_locations(self):
    plt.figure(figsize=(12, 6))
    top_pickups = self.df['PULocationID'].value_counts().nlargest(10)
    sns.barplot(x=top_pickups.index, y=top_pickups.values, palette='viridis')
    plt.title('Top 10 Pickup Locations')
    plt.xlabel('Pickup Location ID')
    plt.ylabel('Number of Trips')
    plt.show()

  def plot_top_dropoff_locations(self):
    plt.figure(figsize=(12, 6))
    top_dropoffs = self.df['DOLocationID'].value_counts().nlargest(10)
    sns.barplot(x=top_dropoffs.index, y=top_dropoffs.values, palette='magma')
    plt.title('Top 10 Dropoff Locations')
    plt.xlabel('Dropoff Location ID')
    plt.ylabel('Number of Trips')
    plt.show()









In [ ]:
Taxi = Taxitrip('Taxi_Trip_Data.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Taxi_Trip_Data.csv'

In [ ]:
Taxi.get_data()

In [ ]:
Taxi.show_info()

In [ ]:
Taxi.fill_nan().isnull().sum()

In [ ]:
Taxi.calculate_trip_duration()

In [ ]:
Taxi.payment_type()

In [ ]:
Taxi.longest_trip_distance()

In [ ]:
Taxi.shortest_trip_distance()

In [ ]:
Taxi.most_popular_pickup_location()

In [ ]:
Taxi.most_popular_dropoff_location()

In [ ]:
Taxi.most_popular_payment_type()

In [ ]:
Taxi.maximum_tip_amount()

In [ ]:
Taxi.average_trip_duration()

In [ ]:
Taxi.average_trip_distance()

In [ ]:
Taxi.average_fare_amount()

In [ ]:
Taxi.average_tip_amount()

In [ ]:
Taxi.average_total_amount()

In [ ]:
Taxi.average_congestion_surcharge()

In [ ]:
Taxi.average_mta_tax()

In [ ]:
Taxi.common_trip()

In [ ]:
Taxi.drop_loc_highest_fare()

In [ ]:
Taxi.correlation()

In [ ]:
Taxi.fraud_trips()

In [ ]:
Taxi.plot_trip_distance_distribution()

In [ ]:
Taxi.plot_fare_amount_distribution()

In [ ]:
Taxi.plot_payment_type_counts()

In [ ]:
Taxi.plot_trip_duration_vs_fare()

In [ ]:
Taxi.plot_tip_amount_distribution()

In [ ]:
Taxi.plot_top_pickup_locations()

In [ ]:
Taxi.plot_top_dropoff_locations()

In [ ]:
for col,dtypes in Taxi.get_data().dtypes.items():
  print(col,dtypes)

In [ ]:
def infer_sql_types(dtype):
  if pd.api.types.is_integer_dtype(dtype):
    return 'INT'
  elif pd.api.types.is_float_dtype(dtype):
    return 'FLOAT'
  elif pd.api.types.is_datetime64_any_dtype(dtype):
    return 'TIMESTAMP'
  elif pd.api.types.is_bool_dtype(dtype):
    return 'BOOLEAN'
  else:
    return 'Text'
table_name = 'Taxi_Data'
columns = Taxi.get_data().dtypes
sql_columns = ",\n ".join([f'"{col}"{infer_sql_types(dtype)}' for col, dtype in columns.items()])

create_table_query = f"CREATE TABLE {table_name} (\n{sql_columns}\n);"
print(create_table_query)

In [ ]:
import psycopg2
import pandas as pd

df = Taxi.get_data()
df = df.where(pd.notnull(df), None)  # Replace NaNs with None for PostgreSQL

table_name = 'Taxi_Data'

try:
    conn = psycopg2.connect(
        dbname='postgres',
        user='postgres',
        password='postgres',
        host='127.0.0.1',
        port='5432'
    )
    cur = conn.cursor()

    for _, row in df.iterrows():
        values = tuple(row)
        placeholders = ', '.join(['%s'] * len(values))
        insert_query = f'INSERT INTO "{table_name}" VALUES ({placeholders})'
        cur.execute(insert_query, values)

    conn.commit()
    print("Data inserted successfully!")

except Exception as e:
    print(f"Insert failed: {e}")

finally:
    if 'cur' in locals() and not cur.closed:
        cur.close()
    if 'conn' in locals() and not conn.closed:
        conn.close()



In [ ]:
Taxi.get_data().to_csv('Cleaned_Taxi_Trip_Data.csv', index= 'False')